# Final code

## [Link for CF Solution](https://www.overleaf.com/4679963215kkhxcqqhzrxh)


## Analysis based on delta coefficients

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
import os

# Parametres

In [31]:
N2, NR, N1, P1 = 1.0, 1.0, 10.0, 10.0
P2_max, PR_max = 10.0, 10.0
tau = 0.25

# Functions

In [32]:
def calculate_A(H11): 
    return ((((H11**2)*P1)/(((1+(((H11**2)*P1)/(N1)))**(1-tau))-1))-N1)    

def C(x): 
    return (1/2*np.log2(1+x))

def mult_res_v2(F, hR2, h12, h1R, hR1, h22, h2R, h21, h11):
  # calculer BF ou SA pour plusieurs echantillons 
  all_res = np.zeros((hR2.shape[0], 3))
  for i in range (hR2.shape[0]):
        
        all_res[i][0] += F(hR2[i], h12[i], h1R[i], hR1[i], h22[i], h2R[i], h21[i], h11[i])[0]
        all_res[i][1] += F(hR2[i], h12[i], h1R[i], hR1[i], h22[i], h2R[i], h21[i], h11[i])[1]
        all_res[i][2] += F(hR2[i], h12[i], h1R[i], hR1[i], h22[i], h2R[i], h21[i], h11[i])[2]

  return all_res
 

def Delta(HR2, H12, H1R, HR1, H22, H2R, H21, H11):
   
    #A = (((H11**2*P1)/(((1+((H11**2*P1)/(N1)))**(1-tau))-1))-N1)    
    A = calculate_A(H11)
    N2_tilde = ((H12**2)*P1+N2)
    NR_tilde = ((H1R**2)*P1+NR)
    P_Z = ((H12*H1R*P1)/(np.sqrt(N2_tilde*NR_tilde)))
    K1 = ((H2R**2)*N2_tilde+(H22**2)*NR_tilde-2*H2R*H22*P_Z*np.sqrt(N2_tilde*NR_tilde))
    K2 = ((1-P_Z**2)*NR_tilde*N2_tilde)
    
    C1 = K1*(HR1**2)*((H22**2)*(HR1**2)-(HR2**2)*(H21**2))
    C2 = K1*(HR2**2)*(H21**2)*A-2*(HR1**2)*(H22**2)*K1*A-(HR1**2)*(H22**2)*(H21**2)*K2
    C3 = (H22**2)*K1*(A**2)+(H22**2)*(H21**2)*K2*A
    C4 = K2*(HR2**2)*(H21**2)**2-N2_tilde*K1*(H21**2)*(HR1**2)
    C5 = N2_tilde*(H21**2)*(K1*A+K2*(H21**2))
    D = C1**2*C5**2-C1*C4*(C2*C5-C3*C4)
    
    return C1,C2,C3,C4,C5,D

def f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr, P2):
    # f en fonction de PR et P2
    A = calculate_A(H11)     

    N2_tilde = ((H12**2)*P1+N2)
    NR_tilde = ((H1R**2)*P1+NR)
    P_Z = ((H12*H1R*P1)/(np.sqrt(N2_tilde*NR_tilde)))
    K1 = ((H2R**2)*N2_tilde+(H22**2)*NR_tilde-2*H2R*H22*P_Z*np.sqrt(N2_tilde*NR_tilde))
    K2 = ((1-P_Z**2)*NR_tilde*N2_tilde)
    #res = (K1*HR2**2*P2*Pr+H22**2*P2*K1*P2+H22**2*P2*K2)/(K2*HR2**2*Pr+N2_tilde*K1*P2+N2_tilde*K2)
    res = (K1*(HR2**2)*P2*Pr+(H22**2)*P2*(K1*P2+K2))/(K2*(HR2**2)*Pr+N2_tilde*K1*P2+N2_tilde*K2)

    
    return res

def f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x):
    'fonction depend seulment de Pr (x apres changement de variable) '
    # f en fonction de PR 
    A = calculate_A(H11)
    N2_tilde = ((H12**2)*P1+N2)
    NR_tilde = ((H1R**2)*P1+NR)
    P_Z = ((H12*H1R*P1)/(np.sqrt(N2_tilde*NR_tilde)))
    K1 = ((H2R**2)*N2_tilde+(H22**2)*NR_tilde-2*H2R*H22*P_Z*np.sqrt(N2_tilde*NR_tilde))
    K2 = ((1-P_Z**2)*NR_tilde*N2_tilde)


    C1 = K1*(HR1**2)*((H22**2)*(HR1**2)-(HR2**2)*(H21**2))
    C2 = K1*(HR2**2)*(H21**2)*A-2*(HR1**2)*(H22**2)*K1*A-(HR1**2)*(H22**2)*(H21**2)*K2
    C3 = (H22**2)*K1*(A**2)+(H22**2)*(H21**2)*K2*A
    C4 = K2*(HR2**2)*(H21**2)**2-N2_tilde*K1*(H21**2)*(HR1**2)
    C5 = N2_tilde*(H21**2)*(K1*A+K2*(H21**2))

    f = (C1*x**2+C2*x+C3)/(C4*x+C5)
    
    return f

# Bruteforce

In [42]:
def BF_V2(HR2, H12, H1R, HR1, H22, H2R, H21, H11):
    ''' Bruteforce '''
    Pr = np.linspace(0.0, PR_max, 1000)
    P2 = np.linspace(0.0, P2_max, 1000)

    B,C = np.meshgrid(Pr, P2)
              
    A = calculate_A(H11)     
    
    # if QoS constraint respected
    mask = B <= (A-(H21**2)*C)/(HR1**2)
    #mask2 = C <= (A-(HR1**2)*B)/(H21**2)

    B = B[mask]
    C = C[mask]
    
    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, B, C)

    ind = np.argmax(SNR)

    SNR_opt, pr_opt, ps_opt = SNR[ind], B[ind], C[ind]
    
    #P2_opt = (A-(HR1**2)*pr_opt)/(H21**2)
    
    #SNR_opt = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, pr_opt, ps_opt)
    #SNR_opt = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, pr_opt, ps_opt)


    
    return pr_opt, ps_opt, SNR_opt

In [34]:
def f_domain(HR2, H12, H1R, HR1, H22, H2R, H21, H11):
    #HR1, H11, H2R, H1R, H22, HR2, H21, H12
    A = calculate_A(H11)
    if ( (A/(HR1**2)) < PR_max and (A/(H21**2)) < P2_max ) :  # H1
        
        x_min, x_max = 0.0, A/(HR1**2)        

    elif ( (A/(HR1**2)) < PR_max and (A/(H21**2)) > P2_max ):# H2
        
        x_min, x_max =  (A-(H21**2)*P2_max)/(HR1**2), A/(HR1**2)

    elif ( (A/(HR1**2)) > PR_max and (A/(H21**2)) < P2_max ):# H3
        
        x_min, x_max =  0.0 , PR_max
    elif ( (A/(HR1**2)) > PR_max and (A/(H21**2)) > P2_max ) and P2_max*(H21**2) + PR_max*(HR1**2) > A: 
        
        x_min, x_max =  (A-(H21**2)*P2_max)/(HR1**2), PR_max
        
    return x_min, x_max, A

def CF_V4(HR2, H12, H1R, HR1, H22, H2R, H21, H11):
    '''
    Version Finale
    '''
        
    A = calculate_A(H11)
    #x_min, x_max = f_domain(HR2, H12, H1R, HR1, H22, H2R, H21, H11)
    if ( (A/(HR1**2)) < PR_max and (A/(H21**2)) < P2_max ) :  # H1

        x_min, x_max = 0.0, A/(HR1**2)        

    elif ( (A/(HR1**2)) < PR_max and (A/(H21**2)) > P2_max ):# H2

        x_min, x_max =  (A-(H21**2)*P2_max)/(HR1**2), A/(HR1**2)

    elif ( (A/(HR1**2)) > PR_max and (A/(H21**2)) < P2_max ):# H3

        x_min, x_max =  0.0 , PR_max
        
    elif ( (A/(HR1**2)) > PR_max and (A/(H21**2)) > P2_max ) and P2_max*(H21**2) + PR_max*(HR1**2) > A: #H4

        x_min, x_max =  (A-(H21**2)*P2_max)/(HR1**2), PR_max
        
    C1, C2, C3, C4, C5, delta = Delta(HR2, H12, H1R, HR1, H22, H2R, H21, H11)

    try :
        if C1*C4>0:
            #print('C1*C4>0')
            
            if delta > 0 :
                x_1 = (-(C1*C5)-np.sqrt(delta))/(C1*C4) 
                x_2 = (-(C1*C5)+np.sqrt(delta))/(C1*C4) 
                if (x_1 <= x_min <= x_max <= x_2): 
                    Pr_opt = x_min 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt,P2_opt)
                elif (x_1<=x_min<=x_2<=x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_min)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)): 
                    Pr_opt = x_min 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt,P2_opt)
                      
                elif x_1 >= x_max :
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
               
                elif x_2 <= x_max :
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)

                elif  (x_1 <= x_min <= x_2 <= x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_min)):
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                    
                elif (x_min <= x_1 <= x_2<=x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_1)): 
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)

                elif (x_min <= x_1 <= x_max <=x_2) :
                    Pr_opt = x_1
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                elif (x_min<=x_1<=x_2<=x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_1)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)):
                    Pr_opt = x_1
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


            elif delta == 0 :
                
                x_0 = -C5/C4
                Pr_opt = x_max
                P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                
            else :
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                    


        elif C1*C4 == 0.0 : 
            #print('C1*C4 == 0')
            x_ = -(C2*C5-C3*C4)/(2*C1*C5)

            if (C1*C5)>0.0 :
                
                if (C2*C5-C3*C4)>0.0 :

                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)

                else : 

                    if x_ < x_min : 

                        Pr_opt = x_max
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)

                    elif (x_min <= x_ <= x_max) : 

                        if f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_min)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max) :
                            
                            Pr_opt = x_min
                            P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                            SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


                        else : 
                            
                            Pr_opt = x_max
                            P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                            SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


                    elif x_ > x_max :

                        Pr_opt = x_min
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


            elif (C1*C5) == 0.0 : 
                #print('C1*C5')
                if (C2*C5-C3*C4) > 0.0 :
                    
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)

                else :
                    
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


            else :
                if (C2*C5-C3*C4)>0.0 :
                    #print((C2*C5-C3*C4))
                    if x_ < x_min :
                        
                        Pr_opt = x_min
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


                    elif (x_min <= x_ <= x_max):
                        
                        Pr_opt = x_
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)

                    elif x_ > x_max :
                        
                        Pr_opt = x_max  #     x_    
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                
                else : # (C2*C5-C3*C4) < 0 ...
                        Pr_opt = x_min        
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


        else :
            #print('C1*C4<0')

            if delta > 0.0 :
                #print('delta>0')
                x_1 = (-(C1*C5)-np.sqrt(delta))/(C1*C4) 
                x_2 = (-(C1*C5)+np.sqrt(delta))/(C1*C4)
                
                if x_1 <= x_min  : 
                    
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                
                elif (x_min<=x_2<=x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_min)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11,x_max)): 
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


                elif  x_max <= x_2:
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                elif  (x_min<=x_2<=x_max<=x_1 and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11,x_min)):
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)


                elif (x_min <= x_1 <= x_max) :
                    
                    Pr_opt = x_1
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                elif (x_min <= x_2 <= x_max <= x_1 and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11,x_min)) :
                    
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                elif (x_2 <= x_min <= x_max <= x_1) :
                    
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
               
                
            elif delta == 0 :
                #print('delta=0')


                x_0 = -C5/C4
                Pr_opt = x_min
                P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                
            else :
                #print('delta<0')
                Pr_opt = x_min
                P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                SNR = f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, Pr_opt, P2_opt)
                

     

    except UnboundLocalError:
        
        Pr_opt, P2_opt, SNR = PR_max, P2_max ,f_obj(HR2, H12, H1R, HR1, H22, H2R, H21, H11, PR_max, P2_max)
        #print('[H5]-[H6]')
    
    
    return Pr_opt, P2_opt, SNR
            
    

# Analytic soluton

# Phase test

## Dataset creation

In [35]:
Nbr = 10000
pos_min = 0 # min position in cell
pos_max = 10 # max position in cell

#-----------------------------------------------------------------------------------------------------------#
#-----------------------------------------------------------------------------------------------------------#

def channel_gain_with_gaussian_fading(d, sigma=7, alpha=3):  # channel gain model
  """
      channel gain model.
      Args:
         d: distance between source and destination
         sigma: gaussian fading sigma
         alpha: is the path loss factor
      Returns:
        channel gain
  """    
  s = np.random.normal(0.0, sigma, d.shape[0])
  h = s/np.sqrt(1.0 + np.power(d, alpha))

  return h

def data_filter(HR2, H12, H1R, HR1, H22, H2R, H21, H11):
  s1 = 10**(-40/10)/10
  s = np.sqrt(s1)
  mask = np.all(np.stack([HR2, H12, H1R, HR1, H22, H2R, H21, H11], axis=1)>=s, axis=1)
  return HR2[mask], H12[mask], H1R[mask], HR1[mask], H22[mask], H2R[mask], H21[mask], H11[mask]


#-----------------------------------------------------------------------------------------------------------#
#-----------------------------------------------------------------------------------------------------------#

U_1 = np.random.uniform(pos_min, pos_max, (Nbr, 2))
D_1 = np.random.uniform(pos_min, pos_max, (Nbr, 2))
U_2 = np.random.uniform(pos_min, pos_max, (Nbr, 2))
D_2 = np.random.uniform(pos_min, pos_max, (Nbr, 2))
R  = 5 * np.ones((Nbr, 2))

d_11 = np.linalg.norm(U_1 - D_1, axis=1)
d_12 = np.linalg.norm(U_1 - D_2, axis=1)
d_1R = np.linalg.norm(U_1 - R  , axis=1)
d_21 = np.linalg.norm(U_2 - D_1, axis=1)
d_22 = np.linalg.norm(U_2 - D_2, axis=1)
d_2R = np.linalg.norm(U_2 - R  , axis=1)
d_R1 = np.linalg.norm(R   - D_1, axis=1)
d_R2 = np.linalg.norm(R   - D_2, axis=1)

var  = 7

h_R2 = channel_gain_with_gaussian_fading(d_R2, var)
h_12 = channel_gain_with_gaussian_fading(d_12, var)
h_1R = channel_gain_with_gaussian_fading(d_1R, var)
h_R1 = channel_gain_with_gaussian_fading(d_R1, var)
h_22 = channel_gain_with_gaussian_fading(d_22, var)
h_2R = channel_gain_with_gaussian_fading(d_2R, var)
h_21 = channel_gain_with_gaussian_fading(d_21, var)
h_11 = channel_gain_with_gaussian_fading(d_11, var)



# Run CF Code

In [36]:
CF_res = mult_res_v2(CF_V4, h_R2, h_12, h_1R, h_R1, h_22, h_2R, h_21, h_11)


# Run Bruteforce Code

In [37]:
BF_res = mult_res_v2(BF_V2, h_R2, h_12, h_1R, h_R1, h_22, h_2R, h_21, h_11)

# Results

In [38]:
np.size(np.where(BF_res[:, 2] > CF_res[:, 2]))
#np.where(BF_res[:, 2] > CF_res[:, 2])

0

In [10]:
# Faux !! np.size(np.where(h_21**2*CF_res[:, 1]+h_R1**2*CF_res[:, 0] > calculate_A(h_11)))

#CF_V4(h_R2[4339], h_12[4339], h_1R[4339], h_R1[4339], h_22[4339], h_2R[4339], h_21[4339], h_11[4339])
#h_21[indx_of_max]**2*CF_res[indx_of_max, 1]+h_R1[indx_of_max]**2*CF_res[indx_of_max, 0] > calculate_A(h_11_n[indx_of_max])


In [39]:

Pr_distance_max = np.max(np.abs(CF_res[:, 0] - BF_res[:, 0]))
P2_distance_max = np.max(np.abs(CF_res[:, 1] - BF_res[:, 1]))
SNR_distance_max = np.max(np.abs(CF_res[:, 2] - BF_res[:, 2]))
distance_SNR = np.abs(CF_res[:, 2] - BF_res[:, 2])
#distance_SNR

In [40]:

indx_of_max = np.where(np.max(distance_SNR)==distance_SNR)
np.max(distance_SNR)

1.0154347411735927

In [41]:
print("PR_SA :",CF_res[indx_of_max, 0])
print("PR_BF :",BF_res[indx_of_max, 0])
print("P2_SA :",CF_res[indx_of_max, 1])
print("P2_BF :",BF_res[indx_of_max, 1])
print("SNR_SA :",CF_res[indx_of_max, 2])
print("SNR_BF :",BF_res[indx_of_max, 2])

PR_SA : [[0.]]
PR_BF : [[0.06006006]]
P2_SA : [[0.26685174]]
P2_BF : [[0.26026026]]
SNR_SA : [[41.12131901]]
SNR_BF : [[40.10588427]]


In [43]:
# changement de l'echelle de la bruteforce pour verifier ce cas (1000 a 10000)
S = BF_V2(h_R2[indx_of_max], h_12[indx_of_max], h_1R[indx_of_max], h_R1[indx_of_max], h_22[indx_of_max], h_2R[indx_of_max], h_21[indx_of_max], h_11[indx_of_max])
# recalcul de la distance
np.abs(S[2] - CF_res[indx_of_max, 2])

array([[0.12711753]])

In [18]:
#np.set_pxrintoptions(formatter={'float': lambda x : '{0:0.2f}'.format(x)})
print('Analytic Solution :')
print('   PR    P2   SNR')
print(np.array2string(CF_res, precision=2, separator=',',
                       suppress_small=True))

print('--------------------------------------------------')

print('Brute Force : ')
print('   PR    P2   SNR')
print(np.array2string(BF_res, precision=2, separator=',', suppress_small=True))

Analytic Solution :
   PR    P2   SNR
[[10.  , 5.15, 1.02],
 [ 0.  , 1.77, 0.22],
 [ 0.  , 5.03, 0.16],
 ...,
 [10.  , 5.07,14.01],
 [10.  , 0.08, 0.  ],
 [10.  , 6.3 , 1.16]]
--------------------------------------------------
Brute Force : 
   PR    P2   SNR
[[10.  , 5.15, 1.02],
 [ 0.06, 1.76, 0.22],
 [ 0.  , 5.03, 0.16],
 ...,
 [10.  , 5.07,13.99],
 [10.  , 0.07, 0.  ],
 [10.  , 6.29, 1.16]]


In [47]:

Pr_distance_max = np.max(np.abs(CF_res[:, 0] - BF_res[:, 0]))
P2_distance_max = np.max(np.abs(CF_res[:, 1] - BF_res[:, 1]))
SNR_distance_max = np.max(np.abs(CF_res[:, 2] - BF_res[:, 2]))

np.max(CF_res[:, 1])

10.000000000485224

In [45]:
CF_res_pd = pd.DataFrame({'Pr': CF_res[:, 0], 'P2': CF_res[:, 1], 'SNR': CF_res[:, 2] })
CF_res_pd.describe()

,Pr,P2,SNR
count,10000.000000,10000.000000,1.000000e+04
mean,5.279746,7.471945,8.597277e+00
std,4.318922,3.714079,5.538911e+01
min,0.000000,0.006965,1.525856e-07
25%,0.569573,4.124612,8.799407e-02
50%,4.878257,10.000000,5.491686e-01
75%,10.000000,10.000000,2.572905e+00
max,10.000000,10.000000,1.796350e+03


In [46]:

BF_res_pd = pd.DataFrame({'Pr': BF_res[:, 0], 'P2': BF_res[:, 1], 'SNR': BF_res[:, 2] })
BF_res_pd.describe()

,Pr,P2,SNR
count,10000.000000,10000.000000,1.000000e+04
mean,5.297651,7.469982,8.595267e+00
std,4.302517,3.714411,5.538563e+01
min,0.000000,0.010010,1.521265e-07
25%,0.620621,4.134134,8.790783e-02
50%,4.934935,10.000000,5.488385e-01
75%,10.000000,10.000000,2.571248e+00
max,10.000000,10.000000,1.796350e+03


# Results

In [132]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_true, y_predicted = CF_res[:,2].astype("float"), BF_res[:,2].astype("float")

##  Mean Absolute Error (SNR)

In [133]:
mean_absolute_error(y_true, y_predicted)

0.003648147975755582

##  Mean Squared Error (SNR)

In [134]:
mean_squared_error(y_true, y_predicted)

0.00471967919039298